# todo
- multiprocessing


In [1]:
from collections import defaultdict
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import feature
from PIL import Image as IMG
import numpy as np
import pandas as pd 
import operator
import cv2
import os 
import gc
from IPython.core.display import HTML 
from IPython.display import Image
from joblib import Parallel, delayed

from myutils import timer

# global vars
## change this path depends on envs
images_path = '../input/ants/'

## const.
features_path = '../features/'



In [2]:
# define functions
def check_imgpath(img):
    return os.path.isfile(images_path + img)

def load_image(img, usecv2=False):
    path = images_path + img
    try:
        im = cv2.imread(path) if usecv2==True else IMG.open(path)
    except Exception as e:
        print('Cannot open img: ', images_path + img)    
    return im

def crop_horizontal(im):
    # cut the images into two halves as complete average may give bias results
    size = im.size
    halves = (size[0]/2, size[1]/2)
    im1 = im.crop((0, 0, size[0], halves[1]))
    im2 = im.crop((0, halves[1], size[0], size[1]))
    return im1, im2

def color_analysis(img):
    # obtain the color palatte of the image 
    palatte = defaultdict(int)
    for pixel in img.getdata():
        palatte[pixel] += 1

    # sort the colors present in the image 
    sorted_x = sorted(palatte.items(), key=operator.itemgetter(1), reverse = True)
    light_shade, dark_shade, shade_count, pixel_limit = 0, 0, 0, 25
    for i, x in enumerate(sorted_x[:pixel_limit]):
        if all(xx <= 20 for xx in x[0][:3]): ## dull : too much darkness 
            dark_shade += x[1]
        if all(xx >= 240 for xx in x[0][:3]): ## bright : too much whiteness 
            light_shade += x[1]
        shade_count += x[1]
        
    light_percent = round((float(light_shade)/shade_count)*100, 2)
    dark_percent = round((float(dark_shade)/shade_count)*100, 2)
    return light_percent, dark_percent

def perform_color_analysis_black(img):
    if check_imgpath(img) ==False:
        return -1

    im = load_image(img)
    im1, im2 = crop_horizontal(im)

    try:
        light_percent1, dark_percent1 = color_analysis(im1)
        light_percent2, dark_percent2 = color_analysis(im2)
    except Exception as e:
        print('Calculation error')
        return None

    dark_percent = (dark_percent1 + dark_percent2)/2 
    return dark_percent

def perform_color_analysis_white(img):
    if check_imgpath(img) ==False:
        return -1

    im = load_image(img)
    im1, im2 = crop_horizontal(im)

    try:
        light_percent1, dark_percent1 = color_analysis(im1)
        light_percent2, dark_percent2 = color_analysis(im2)
    except Exception as e:
        print('Calculation error')
        return None

    light_percent = (light_percent1 + light_percent2)/2 
    return light_percent

def average_pixel_width(img):
    if check_imgpath(img) ==False:
        return -1

    im = load_image(img)    
    im_array = np.asarray(im.convert(mode='L'))
    edges_sigma1 = feature.canny(im_array, sigma=3)
    apw = (float(np.sum(edges_sigma1)) / (im.size[0]*im.size[1]))
    return apw*100

def get_dominant_color(img):
    if check_imgpath(img) ==False:
        return [-1, -1, -1]

    im = load_image(img, usecv2=True)
    arr = np.float32(im)
    pixels = arr.reshape((-1, 3))

    n_colors = 5
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, centroids = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)

    palette = np.uint8(centroids)
    quantized = palette[labels.flatten()]
    quantized = quantized.reshape(im.shape)

    dominant_color = palette[np.argmax(np.unique(labels, return_counts=True)[1: ])]
    return dominant_color

def get_average_color(img):
    if check_imgpath(img) ==False:
        return [-1, -1, -1]

    im = load_image(img, usecv2=True)
    average_color = [im[:, :, i].mean() for i in range(im.shape[-1])]
    return average_color

def get_size(img):
    if check_imgpath(img) ==False:
        return -1

    filename = images_path + img
    st = os.stat(filename)
    return st.st_size

def get_dimensions(img):
    if check_imgpath(img) ==False:
        return [-1, -1]

    im = load_image(img)
    img_size = im.size
    return img_size

def get_blurrness_score(img):
    if check_imgpath(img) ==False:
        return -1

    im = load_image(img, usecv2=True)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(im, cv2.CV_64F).var()
    return fm

def get_imagefeatures(features, imgcol, prefix=''):
    with timer('Image features extraction'):
        prefix = prefix + '_'
        features[prefix+'dullness'] = features[imgcol].apply(perform_color_analysis_black)
        features[prefix+'whiteness'] = features[imgcol].apply(perform_color_analysis_white)
        features[prefix+'average_pixel_width'] = features[imgcol].apply(average_pixel_width)
        features[prefix+'dominant_color'] = features[imgcol].apply(get_dominant_color)
        features[prefix+'dominant_red'] = features[prefix+'dominant_color'].apply(lambda x: x[0]) / 255
        features[prefix+'dominant_green'] = features[prefix+'dominant_color'].apply(lambda x: x[1]) / 255
        features[prefix+'dominant_blue'] = features[prefix+'dominant_color'].apply(lambda x: x[2]) / 255
        features.drop(prefix+'dominant_color', axis=1, inplace=True)
        features[prefix+'average_color'] = features[imgcol].apply(get_average_color)
        features[prefix+'average_red'] = features[prefix+'average_color'].apply(lambda x: x[0]) / 255
        features[prefix+'average_green'] = features[prefix+'average_color'].apply(lambda x: x[1]) / 255
        features[prefix+'average_blue'] = features[prefix+'average_color'].apply(lambda x: x[2]) / 255
        features.drop(prefix+'average_color', axis=1, inplace=True)
        features[prefix+'image_size'] = features[imgcol].apply(get_size)
        features[prefix+'temp_size'] = features[imgcol].apply(get_dimensions)
        features[prefix+'width'] = features[prefix+'temp_size'].apply(lambda x : x[0])
        features[prefix+'height'] = features[prefix+'temp_size'].apply(lambda x : x[1])
        features.drop(prefix+'temp_size', axis=1, inplace=True)
        features[prefix+'blurrness'] = features[imgcol].apply(get_blurrness_score)
    
    gc.collect()
    return features

def save_features(features, filename):
    filepath = features_path + filename
    features.to_feather(filepath)


In [4]:
# Example: Calculate features
imgs = os.listdir(images_path)
features = pd.DataFrame()
features['imagepath'] = imgs
features['imagepath'][0] = ""

features = get_imagefeatures(features, imgcol='imagepath', prefix='debug')

gc.collect()
features.head()

[Image features extraction] done in 35 s


,imagepath,debug_dullness,debug_whiteness,debug_average_pixel_width,debug_dominant_red,debug_dominant_green,debug_dominant_blue,debug_average_red,debug_average_green,debug_average_blue,debug_image_size,debug_width,debug_height,debug_blurrness
0,,-1.0,-1.000,-1.000000,-0.003922,-0.003922,-0.003922,-0.003922,-0.003922,-0.003922,-1,-1,-1,-1.000000
1,11381045_b352a47d8c.jpg,0.0,0.000,1.934535,0.074510,0.415686,0.337255,0.196553,0.399769,0.424831,36120,500,333,891.795181
2,119785936_dd428e40c3.jpg,0.0,3.235,2.565868,0.094118,0.192157,0.203922,0.345592,0.450838,0.500565,57087,500,334,193.591465
3,147542264_79506478c2.jpg,0.0,4.220,2.411255,0.184314,0.584314,0.243137,0.287481,0.561964,0.439412,54044,400,462,132.693869
4,17081114_79b9a27724.jpg,0.0,7.530,3.899200,0.686275,0.705882,0.705882,0.555366,0.587042,0.596563,94240,375,500,10261.052169


In [ ]:
# Example: Save featuers
features.drop('imagepath', axis=1, inplace=True)
save_features(features, filename='debug.feather')
